In [20]:
import numpy as np

from bokeh.io import push_notebook, output_notebook, show, curdoc
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points

curdoc().theme = 'dark_minimal'

Loading BokehJS ...

# Chapter 1: Transparent Conductors

Transparent conductors are materials that simultaneously possess good electrical conductivity and optical transparency. They are essential for a range of devices including touchscreens, solar cells, and LED displays.

### Instruction:
The purpose of this notebook is to identify values of electron density and mean free time for a material, aiming for its potential use as a transparent conductor. The goal is to estimate the order of magnitude for these two parameters.

#### Initial Setup:
We start by setting up the foundational variables for our calculations:

- **Electron Density (n)**: Represents the number of conduction electrons per unit volume.
  
- **Electron Mass (m) and Charge (e)**: Fundamental properties of an electron.
  
- **Material Resistivity/Conductivity**: Resistivity is a measure of a material's ability to resist the flow of electric current, and its inverse is conductivity.
  
- **Vacuum and Effective Material Permittivity (ε and ε0)**: These values relate to how much a material becomes polarized in response to an external electric field.
  
- **Frequency Range**: The boundaries of frequency over which we'll evaluate the material properties.
  
- **Mean Free Time (τ)**: This is the average time between successive collisions of an electron, and it can be deduced from the relation:
\[$ \sigma = \frac{n e^2 \tau}{m} $\]
where \( \sigma \) is the conductivity.




In [21]:
# Set up constant in SI units
freq_min = 1e+14   #Lower limit of frequency
freq_max = 10e+14    #Upper limit of frequency
dens = 8.47e+28 #Define initial electron density
m = 9.109e-31 #Define electron mass
elec = 1.602e-19 #Define elementary electron charge
rho = 1.56e-8 #Define DC resistivity
sigma0 = 1/rho #Calculate initial DC conductivity from resisitivity
eps = 8.854e-12 #Vacuum permittivity
eps0 = 1.0 #Effective permittivity of the material
N = 5000   #Number of points for the independent variable
tau = m / (dens*rho*elec**2) #Calculate mean free time

In [22]:
# Set up the variable space
freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
w = 2*np.pi*freq #Convert frequency to angular frequency
# Calculate AC conductivity
sigma1 = sigma0 / (1.0 + w**2*tau**2)  #Real part of the conductivity
sigma2 = w*tau*sigma0 / (1.0 + w**2*tau**2)  #imaginary part of the conductivity

#### Calculations:
After defining our variables:

1. **Frequency Spectrum**: We delineate a spectrum of frequency points and then convert these frequencies to their angular counterparts.
  
2. **AC Conductivity**: Using the aforementioned parameters, we compute both the real and imaginary components of AC conductivity.
  
3. **Permittivity**: Given the AC conductivity, we further calculate the material's permittivity.



In [23]:
#calculate permittivity
eps1 = eps0 - sigma2 /(eps*w)
eps2 = sigma1/(eps*w)

#### Visualization:
To visualize our results:

- We set up four plots showcasing the real and imaginary parts of both conductivity and permittivity.
  
- We also introduce an update function, empowering users to interactively modify input parameters (like electron density and resistivity) and immediately observe the resultant changes on the plots.



In [24]:
# Set up conductivity plot
plot = figure(plot_height=200, plot_width=400, title="Real part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max])
plot.xaxis[0].axis_label='Frequency (Hz)'
plot.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
r = plot.line(freq, sigma1, line_width=3, line_alpha=0.6)

plot2 = figure(plot_height=200, plot_width=400, title="Imaginary part of AC conductivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max], y_range=[0.0, 2e+7])
plot2.xaxis[0].axis_label='Frequency (Hz)'
plot2.yaxis[0].axis_label='conductivity (Ohm-1m-1)'
rr = plot2.line(freq, sigma2, line_width=3, line_alpha=0.6, color = 'red')

In [25]:
# Set up permittivity plot
plot3 = figure(plot_height=200, plot_width=400, title="Real part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max])
plot3.xaxis[0].axis_label='Frequency (Hz)'
plot3.yaxis[0].axis_label='Permittivity (F/m)'
rrr = plot3.line(freq, eps1, line_width=3, line_alpha=0.6)

plot4 = figure(plot_height=200, plot_width=400, title= "imaginary part of permittivity",
              tools="pan,reset,save,wheel_zoom",
              x_range=[freq_min, freq_max])
plot4.xaxis[0].axis_label='Frequency (Hz)'
plot4.yaxis[0].axis_label='Permittivity (F/m)'
rrrr = plot4.line(freq, eps2, line_width=3, line_alpha=0.6, color = 'red')

In [26]:
# Set up callbacks to update the plots
def update_data(density_pow, mft_pow):
# Generate the new curve
    dens = 10**density_pow
    tau = 10**mft_pow
    rho = m / (dens*tau*elec**2)
    sigma0 = 1/rho
    freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
    w = 2*np.pi*freq #Convert frequency to angular frequency
    r.data_source.data['y'] = sigma0 / (1.0 + w**2*tau**2)
    rr.data_source.data['y'] = w*tau*sigma0 / (1.0 + w**2*tau**2)
    rrr.data_source.data['y'] = eps0 - rr.data_source.data['y']/(eps*w)
    rrrr.data_source.data['y'] =  r.data_source.data['y']/(eps*w)
    print('DC Conductivity =', f'{sigma0:.3}', 'S/m')

    push_notebook()
    
def update_data_prec(density_pow, mft_pow):
# Generate the new curve
    dp = 1 ###  Replace with your guess of order of magnitude for electron density
    tp = 1 ###  Replace with your guess of order of magnitude for mean free time
    dens = density_pow*10**dp
    tau = mft_pow*10**tp
    rho = m / (dens*tau*elec**2)
    sigma0 = 1/rho
    freq = np.linspace(freq_min, freq_max, N)  #Energy measured in units of chemical potential
    w = 2*np.pi*freq #Convert frequency to angular frequency
    r.data_source.data['y'] = sigma0 / (1.0 + w**2*tau**2)
    rr.data_source.data['y'] = w*tau*sigma0 / (1.0 + w**2*tau**2)
    rrr.data_source.data['y'] = eps0 - rr.data_source.data['y']/(eps*w)
    rrrr.data_source.data['y'] =  r.data_source.data['y']/(eps*w)
    print('DC Conductivity =', sigma0, 'S/m')

    push_notebook()

#### Exploration:

In the ensuing code, sliders for the electron density (`density_pow`) and mean free time (`mft_pow`) allow for dynamic adjustments. The current setup permits changes in the order of magnitude for these parameters. It's important to determine and input suitable ranges for these sliders based on the material or context you're exploring.

**Note**: For perspective, the conductivity of copper at room temperature is approximately \($5.96 \times 10^7 \text{ S/m}$\).

For the curious mind, a variation of the update function, named `update_data_prec`, has been provided. Switching to this function (though not mandatory) equips the sliders with linear ranges. If choosing this route, you'd need to integrate your estimated orders of magnitude for the two parameters into the equation provided. This method can offer enhanced precision for parameter estimations.



In [33]:
den_min_range = 1 #Replace with minimum for the range of electron density order of magnitude (e.g. 2 corresponds to n = 10^2)
den_max_range = 1 #Replace with maximum for the range of electron density order of magnitude
mft_min_range = 1 #Replace with minimum for the range of mean free time (tau) order of magnitude
mft_max_range = 1 #Replace with maximum for the range of mean free time (tau) order of magnitude 

show(column([row(plot,plot2),row(plot3, plot4)]),notebook_handle=True)
interact(update_data, density_pow = (den_min_range, den_max_range, 1), mft_pow = (mft_min_range, mft_max_range, 1))

interactive(children=(IntSlider(value=1, description='density_pow', max=1, min=1), IntSlider(value=1, descript…

<function __main__.update_data(density_pow, mft_pow)>

### Self-Education Questions:

1. Why are transparent conductors crucial in modern electronic devices?
  
2. Explain the relationship between resistivity and conductivity. How do they affect the performance of a conductor?
  
3. What roles do electron density and mean free time play in determining a material's conductivity?
  
4. Why is it important to consider both the real and imaginary parts of conductivity and permittivity?
  
5. How does the mean free time (τ) relate to the overall scattering mechanisms in a material?
  
6. What are the implications of having a high or low permittivity for a transparent conductor?
  
7. How might the frequency of incident light affect the performance of a transparent conductor in an application like solar cells or touchscreens?
  
8. Discuss some real-world applications of transparent conductors. How might the required properties vary between these applications?

9. What other material properties, not discussed here, might be crucial when considering a transparent conductor for a specific application?

10. If given the conductivity of another material (not copper), how would you adjust the notebook to consider this new material?

By pondering and researching answers to these questions, students can deepen their understanding of transparent conductors and the physics underlying their operation.